## Gold Layer
- Gold layer usually has aggregations that can introduce or represents some kpi's that will help business units to make precise decisions.
- this layer is very important, once it can impact directly into the company revenue.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
schema = 'gold'
table = 'brewery_aggregated'
table_full_name = f'{schema}.{table}'
data_lake_target = 'https://storagecasebudin.blob.core.windows.net/api-pipeline/gold/'

In [0]:
df_silver = spark.read.format("delta").load("/mnt/silver/")

In [0]:
# Aggregate the number of breweries per type and state
df_gold = df_silver.groupBy("name", "brewery_type", "state", "country", "city") \
                   .agg(count("id").alias("brewery_count")) \
                   .withColumn("ts_loadtimestamp_gold", current_timestamp())


## 2. Save the aggregated table

In [0]:
df_gold.write \
    .format('delta') \
    .mode('overwrite') \
    .partitionBy("ts_loadtimestamp_gold") \
    .option('path', data_lake_target) \
    .option('overwriteSchema', True) \
    .save()

In [0]:
spark.sql(f'OPTIMIZE budinworkspac.gold.gold_brewery_aggregated')

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de